In [9]:
import os
import sys
import pandas as pd
import openai
import pandas as pd
from glob import glob
from tqdm import tqdm
import json
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KE")

current_path = Path.cwd()
src_dir = current_path.parent
sys.path.append(str(src_dir))
# import annotation methods
from src.labels_generator import (generate_relations,relation_search,resort_relation)

# Load matcher
from src.matcher.core import SimCSE_Matcher
matcher = SimCSE_Matcher(str(src_dir/ 'artifacts/matcher_model'))


In [2]:
prompt = '''Your task is to identify the relation between companies mentioned in the report given in ``` quote.
possible relation - {customer, supplier, financial_trade, nothing}


##Report

```
{sentence}

```



## Rules for `financial_trade`
- Companies mentioned in financing, investing in another company are in financial_trade relation
- Companies buying and selling shares or interest in another company are in financial_trade relation
- Companies mentioned in merger and acquisition trade are in financial_trade relation
- companies involved in purchase assets or related party transactions are in financial_trade relation
- companies involved in collaboration agreement for joint development are in financial_trade relation
- Companies in debtor and creditor relation with each other are in financial_trade relation
- Companies doing a transaction for working capital purpose are in financial_trade relation
- Companies involved in a payment apart of the revenue or sales are in financial_trade relation


## Rules for identifying the customer supplier relation
- Customer companies may be referred to as a source of revenue for supplier companies
- Customer companies may be referred to as companies that accounts for revenues or net sales of supplier companies
- Customer companies may be referred to as companies that accounts for trade balance, accounts receivable or amount payable of supplier companies
- Customer companies may be referred to as companies that assign or transfer manufacturing responsibilities to suppliers
- Customer companies may be referred to as distributors, commercializers, retailers, insurance companies, or similar entities.
- Customer companies could act as a distribution channel. Supplier has original content, and they distribute it via customer companies platform or channel

- Supplier companies depend on customer companies to supply or sale of their product
- Supplier companies may be referred to as companies that gain or take revenues from customers.
- Supplier companies may be referred to as those who own, operate, or manufacture materials and then sell to customer companies
- Supplier companies may be referred to as vendors or providers of services, products, or materials to customer companies



## Rule for nothing relation
- If you do not find any possible relation, then select nothing relation
- If sentence is incomplete and don't have meaning


## Rules to follow for all relations -
- use only information given in the report
- Don't infer anything outside of given context


## output

Return JSON Object contains relations with following -
[{'company_1': <company_name>, 'relation: what <company_1> represent to <company_2>(supplier or customer or financial_trade or nothing),
'company_2': <company_name>}]

'''

In [3]:
# Read data
data = pd.read_excel("llm_conflicts_with_sme.xlsx", index_col=0)

/home/ahmed/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [4]:
replaces = {"sentence": "{sentence}"}
# Replace the keys with values for unified relation direction
relations_map = {"customer": "supplier"}
# Generate list of relations in tuples
output = generate_relations(data[:4], prompt, replaces, relations_map)

Generating relations: 100%|██████████| 4/4 [00:33<00:00,  8.35s/it]


In [10]:
# Resort the sme_relations to unify the relations directions
if not isinstance(output['sme_relations'].iloc[0], list):
    tqdm.pandas(desc="Eval string of list")
    output['sme_relations'] = output['sme_relations'].progress_apply(eval)
tqdm.pandas(desc="Resort sme relations")
output['sme_relations'] = output['sme_relations'].progress_apply(lambda x:\
                          resort_relation((x[0], x[1], x[2]),
                                        relations_map))

Resort sme relations: 100%|██████████| 4/4 [00:00<00:00, 12935.40it/s]


In [11]:
# Search relations and return mask
tqdm.pandas(desc="Search relations")
output['align'] =\
output[['sme_relations', 'relations']]\
.progress_apply(lambda x:
relation_search(
query_relation= x[0],
relations_tuples=x[1],
matcher=matcher,
threshold=0.85,
main_relations=list(relations_map.values()) ),axis=1).to_list()


Search relations: 100%|██████████| 4/4 [00:01<00:00,  2.57it/s]


In [12]:
output

,sentence,Label,explanation,sme_relations,llms_relations,True Label (sme/llm/both/none),Comment on true label,comment on llm mistake (refer explanation col if required),comment to check with vishal,Vishal's Label,Vishal's comments,Additional commens,relations,align
index,,,,,,,,,,,,,,
2,ORBCOMM Inc reports that Systems Inc and inthi...,1,"Based on the definitions provided, we can iden...","[ORBCOMM Inc., supplier, Hub Group Inc.]","[['Value added Solutions', 'supplier', 'Carrie...",both,Both parties agreed that Hub Group Inc is a cu...,NaN,In this sentence Hub Group Inc could be refere...,none,ahmad.alaa@isemantics.ai Shyam Shinde\nAgree t...,NaN,"[[ORBCOMM Inc, nothing, Systems Inc], [ORBCOMM...",False
3,For example HELIX ENERGY SOLUTIONS GROUP Inc s...,1,Based on the information provided in the repor...,"[HELIX ENERGY SOLUTIONS GROUP INC, supplier, P...","[['HELIX ENERGY SOLUTIONS GROUP Inc', 'nothing...",sme,NaN,NaN,NaN,NaN,NaN,NaN,"[[Petrobras, supplier, HELIX ENERGY SOLUTIONS ...",False
8,Of the approximately 58 million square feet of...,1,"Based on the definitions provided, it is not p...","[General Growth Properties, Inc., supplier, Li...","[['GAP', 'nothing', 'Limited Brands'], ['GAP',...",sme,NaN,LLm need to understand the tenants concept,NaN,NaN,NaN,NaN,"[[The GAP, supplier, General Growth Properties...",False
9,"Additionally, the addition of new aircraft typ...",1,"Based on the definitions provided, it is not p...","[Sikorsky, supplier, Bristow Group Inc]",[],sme,NaN,LLM couldn't understand the indirect relation ...,NaN,NaN,NaN,NaN,"[[Bristow Group Inc, nothing, nothing]]",False
